In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import guidance
from functools import partial

from speculative_grammar_backtracking import guide, load_parser
from blendsql import LLMMap, LLMQA, blend
from blendsql.models import TransformersLLM
from blendsql.db import SQLite
from blendsql.utils import fetch_from_hub
from blendsql.grammars.utils import load_blendsql_grammar
from blendsql.nl_to_blendsql import create_system_prompt, FewShot, NLtoBlendSQLArgs, post_process_blendsql

In [2]:
db = SQLite(
    fetch_from_hub("1884_New_Zealand_rugby_union_tour_of_New_South_Wales_1.db")
)
ingredients={LLMQA, LLMMap}
prompt = partial(
    create_system_prompt,
    ingredients=ingredients,
    db=db,
    args=NLtoBlendSQLArgs(
        use_tables=["w"],
        include_db_content_tables=["w"],
        num_serialized_rows=3,
        use_bridge_encoder=True,
    ),
    few_shot_examples=FewShot.hybridqa[:4]
)

In [3]:
model_name_or_path = "HuggingFaceTB/SmolLM-360M"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
parser = load_parser(lark_grammar_str=load_blendsql_grammar(ingredients))

In [4]:
res = guide(
    model,
    tokenizer=tokenizer,
    parser=parser,
    prompt=prompt(question="What was the result of the game in the city located 120 miles west of Sydney?"),
    draft_model=guidance.models.Transformers(
        model_name_or_path, echo=False
    ),
    stop_at=['```', '---'],
    max_grammar_corrections=20,
    max_new_tokens=200,
    temperature=0.0,
    verbose=True,
)
pred_query = post_process_blendsql(
    res.response,
    db=db,
    use_tables=["w", "documents"]
)

Generate Blend SQL given the question , table , and passages to answer the question correctly . 
 Bl end SQL is a supers et of SQL ite , which adds external function calls for information not found within native SQL ite . 
 These external functions should be wrapped in double curly brackets . 
 
 If mapping to a new column still cannot answer the question with valid SQL , turn to an end - to - end solution using the aggregate function : 
 ` {{ LL M QA (' question ', ( bl ends ql )) }} ` 
 Option ally , this function can take an ` options ` argument to restrict its output to an existing SQL column . 
 For example : ` ... WHERE column = {{ LL M QA (' question ', ( bl ends ql ), options =' table :: column ) }} ` 

 
 If question - relevant column ( s ) contents are not suitable for SQL comparisons or calculations , map it to a new column using the scalar function : 
 ` {{ LL MM ap (' question ', ' table :: column ') }} ` 
 
 ON LY use these Blend SQL ingredients if necessary . Answer parts of the question in vanilla SQL , if possible . 
 
 Additionally , we have the table ` documents ` at our disposal , which contains Wikipedia articles providing more details about the values in our table . 
 The ` documents ` table for each database has the same schema : 
 
 CREATE TABLE documents ( 
 " title " TEXT , 
 " content " TEXT 
 ) 
 
 Examples : 
 
 CREATE TABLE " w " ( 
 " index " INT EGER , 
 " name " TEXT , 
 " prov ince " TEXT , 
 " city " TEXT , 
 " year " TEXT , 
 " rem arks " TEXT 
 ) 
 /* 
 3 example rows : 
 SELECT * FROM w LIMIT 3 
 index name province city year remarks 
 0 abd ul ra h man mosque k ab ul province k ab ul 2 0 0 9 largest mosque in af ghan istan 
 1 f riday mosque of k and ah ar k and ah ar province k and ah ar 1 7 5 0 houses the cloak of the is lam ic prophet m uh ammad 
 2 o mar al - far oo q mosque k and ah ar province k and ah ar 2 0 1 4 built on the site that was a popular cinema of k and ah ar . [ 1 ] 
 * / 
 
 Here are some values that may be useful : w . city ( her at ) , documents . title ( her at , fire temple ) 
 Q : Who were the builders of the mosque in Her at with fire temples ? 
 Bl end SQL : 
 {{ 
 L LM QA ( 
 ' Who were the builders of the mosque ? ', 
 ( 
 SELECT documents . title AS ' Building ', documents . content FROM documents 
 JO IN {{ 
 L LM Join ( 
 left _ on =' w :: name ', 
 right _ on =' documents :: title ' 
 ) 
 } } 
 WHERE w . city = ' her at ' AND w . rem arks LI KE '% fire temple % ' 
 ) 
 ) 
 }} 
 
 --- 
 
 CREATE TABLE " w " ( 
 " index " INT EGER , 
 " camp name " TEXT , 
 " c ouncil " TEXT , 
 " location " TEXT , 
 " status " TEXT , 
 " notes " TEXT 
 ) 
 /* 
 3 example rows : 
 SELECT * FROM w LIMIT 3 
 index camp name council location status notes 
 0 camp aqu ila ( formerly camp m au we hu ) connect icut y an kee council candle wood lake , sher man , ct closed located on candle wood lake in sher man , ct . , the camp was sold in 1 9 8 2 along with camp to qu am in g os hen 
 1 camp co chip ian ee br ist ol area council g os hen , ct closed the camp was founded in 1 9 2 8 by the br ist ol area council and was sold after the new brit ain area council and the br ist ol area council were merged in 1 9 7 2 
 2 camp ir ving hous atonic council shel ton , ct closed the camp was located in shel ton in the birch bank area along the hous atonic river . it was closed in 1 9 4 5 and the buildings were ra zed in 1 9 4 8 
 * / 
 
 Q : What is the status of the camp in the town that split from St oning ton in 1 7 2 4 ? 
 Bl end SQL : 
 SELECT status FROM w WHERE location = {{ 
 L LM QA ( 
 ' Which town split from St oning ton in 1 7 2 4 ? ', 
 ( 
 SELECT title , content FROM documents 
 WHERE documents M ATCH ' ston ington ' 
 ), 
 options =' w :: location ' 
 ) 
 }} 
 
 --- 
 
 CREATE TABLE " w " ( 
 " index " INT EGER , 
 " no " INT EGER , 
 " r ider " TEXT , 
 " team " TEXT , 
 " motor cycle " TEXT 
 ) 
 /* 
 3 example rows : 
 SELECT * FROM w LIMIT 3 
 index no rider team motorcyc

In [13]:
print(pred_query)

{{
    LLMQA(
        'What was the result of the game in the city located 120 miles west of Sydney ?',
        (
            SELECT documents."title" AS 'Result', documents."content" FROM documents
            JOIN {{
                LLMJoin(
                    left_on = 'w::rival',
                    right_on = 'documents::title'
                )
            }}
            WHERE w."city" = 'sydney' AND w."rival" = '120 miles west of sydney'
        )
    )
}}

---


In [15]:
smoothie = blend(
    query=pred_query,
    db=db,
    default_model=TransformersLLM(model_name_or_path),
    ingredients=ingredients
)

UnboundLocalError: local variable 'scm' referenced before assignment

In [11]:
smoothie.df

,index,#,date,rival,city,venue,score


In [13]:
print(smoothie.summary())

---------------- SUMMARY ----------------
SELECT * FROM w WHERE "city" = 'sydney' AND "date" = '120'
┌────────────┬────────────────────────────────┬─────────────────┬─────────────────────┐
│   Time (s) │   Values Passed to Ingredients │   Prompt Tokens │   Completion Tokens │
├────────────┼────────────────────────────────┼─────────────────┼─────────────────────┤
│  0.0277853 │                              0 │               0 │                   0 │
└────────────┴────────────────────────────────┴─────────────────┴─────────────────────┘
